In [2]:
import sys
import os
current_dir = os.path.dirname(os.path.abspath('./'))
if not current_dir in sys.path:
    sys.path.append(current_dir)
from combiners import EpochsCombiner
from typing import *
import mne
import tensorflow as tf
import mneflow as mf
import matplotlib.pyplot as plt
import numpy as np
from utils.data_management import dict2str
from lfcnn_vis import plot_patterns
from LFCNN_decoder import SpatialParameters, TemporalParameters, ComponentsOrder, Predictions
import pickle
from utils.machine_learning import one_hot_decoder
import sklearn.metrics as sm
from utils.machine_learning.confusion import ConfusionEstimator

def read_pkl(path: str) -> Any:
    with open(
            path,
            'rb'
        ) as file:
        content = pickle.load(
            file
        )
    return content

2022-03-30 11:22:15.927357: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-30 11:22:15.927396: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
import pandas as pd
from utils.console.colored import warn

class PredictionsParser(object):
    def __init__(
        self,
        y_true: Union[list[int], np.ndarray],
        y_pred: Union[list[int], np.ndarray],
        class_names: Optional[Union[str, list[str]]] = None
    ):
        
        y_true = self.__check_numpy(y_true)
        y_pred = self.__check_numpy(y_pred)
        
        if y_true.shape != y_pred.shape:
            raise ValueError('Predictions and actual values are inconsistent. Actual values shape: {y_true.shape}, predictions shape: {y_pred.shape}')
        
        if len(y_true.shape) != 1:
            y_true = one_hot_decoder(y_true)
            y_pred = one_hot_decoder(y_pred)
        
        self._y_true = y_true
        self._y_pred = y_pred
        
        self._accuracy = sm.accuracy_score(y_true, y_pred)
        
        classes_true = np.unique(self._y_true)
        classes_pred = np.unique(self._y_pred)
        
        if np.any(classes_true != classes_pred):
            warn(f'Warning: Classes are inconsistent.\n\tActual classes: {classes_true}\n\tPredicted classes: {classes_pred}.\nTake actual classes')
            
        del classes_pred
        
        self._classes = classes_true
        self._n_classes = len(classes_true)
        
        if class_names is not None:
            
            if isinstance(class_names, str):
                class_names = class_names.split(' ')
            
            if len(class_names) != self.n_classes:
                raise ValueError(f'Class names and classes are inconsistent: number of classes is {self.n_classes}, but {len(class_names)} names of classes were given')
        else:
            class_names = [f'Class {i}' for i in range(self.n_classes)]
        
        self._class_names = class_names
        
        self._confusion = pd.DataFrame(
            sm.confusion_matrix(self.y_true, self.y_pred),
            index = [f'Actual {class_name}' for class_name in self.class_names],
            columns = [f'Predicted {class_name}' for class_name in self.class_names]
        )
    
    @staticmethod
    def __check_numpy(arr: Union[list, tuple, np.ndarray]):
        if isinstance(arr, np.ndarray):
            return arr
        elif not isinstance(arr, np.ndarray) and isinstance(arr, (list, tuple)):
            return np.array(arr)
        else:
            raise ValueError(f'The given argument must be either a np.ndarray or a list, but {type(arr)} was given')
        
    @property
    def y_true(self):
        return self._y_true
    @y_true.setter
    def y_true(self, value):
        raise AttributeError('Impossible to set y_true directly')
    
    @property
    def y_pred(self):
        return self._y_pred
    @y_pred.setter
    def y_pred(self, value):
        raise AttributeError('Impossible to set y_pred directly')
    
    @property
    def accuracy(self):
        return self._accuracy
    @accuracy.setter
    def accuracy(self, value):
        raise AttributeError('Impossible to set accuracy directly')
    
    @property
    def classes(self):
        return self._classes
    @classes.setter
    def classes(self, value):
        raise AttributeError('Impossible to set classes directly')
    
    @property
    def n_classes(self):
        return self._n_classes
    @n_classes.setter
    def n_classes(self, value):
        raise AttributeError('Impossible to set number of classes directly')
    
    @property
    def class_names(self):
        return self._class_names
    @class_names.setter
    def class_names(self, value):
        raise AttributeError('Impossible to set names for classes directly')
    
    @property
    def confusion(self):
        return self._confusion
    @confusion.setter
    def confusion(self, value):
        raise AttributeError('Impossible to set confusion matrix directly')
    
    def summary(self, *args: str):
        df = self.confusion.copy()
        summary = pd.DataFrame(columns = self.class_names)
        summary.loc['Total'] = [df[column].sum() for column in df.columns]
        summary.loc['Accuracy'] = [None for _ in range(self.n_classes)]
        summary.loc['Specificity'] = [None for _ in range(self.n_classes)]
        summary.loc['Sensitivity'] = [None for _ in range(self.n_classes)]
        
        ec = self.estimate_confusion()
        
        args = list(args)
        for i, arg in enumerate(args):
            if not isinstance(arg, tuple):
                args[i] = arg, arg
        
        for arg_value, arg_name in args:
            if hasattr(ec[self.class_names[0]], arg_value):
                summary.loc[arg_name] = [None for _ in range(self.n_classes)]
            else:
                warn(f'WARNING: the {arg_value} property was not found in the confusion evaluator, so it was ignored')
                args.remove((arg_value, arg_name))
            
        
        for i, class_name in enumerate(self.class_names):
            summary[class_name].loc['Accuracy'] = ec[class_name].acc
            summary[class_name].loc['Specificity'] = ec[class_name].spec
            summary[class_name].loc['Sensitivity'] = ec[class_name].sens
            
            for arg_value, arg_name in args:
                summary[class_name].loc[arg_name] = getattr(ec[class_name], arg_value)
            
        return summary
    
    def estimate_confusion(self):
        
        return {
            class_name: ConfusionEstimator(
                # tp, tn, fp, fn
                self.confusion[self.confusion.columns[i]][self.confusion.index[i]],
                self.confusion[
                    list(self.confusion.columns[:i]) + list(self.confusion.columns[i+1:])
                    ].loc[
                        list(self.confusion.index[:i]) + list(self.confusion.index[i+1:])
                ].sum().sum(),
                self.confusion[self.confusion.columns[i]].loc[
                    list(self.confusion.index[:i]) + list(self.confusion.index[i+1:])
                ].sum(),
                self.confusion[
                    list(self.confusion.columns[:i]) + list(self.confusion.columns[i+1:])
                ].loc[self.confusion.index[i]].sum()
            )
            for i, class_name in enumerate(self.class_names)
        }



In [20]:
# subject_name = 'Az_Mar_05'
# subject_name = 'Ga_Fed_06'
subject_name = 'Ku_EL_09'

perf_tables_path = '../Source/perf_tables/'
from utils.storage_management import check_path
check_path(perf_tables_path)

for subject_name in os.listdir(f'../Source/Subjects/'):

    path = f'../Source/Subjects/{subject_name}/LFCNN'

    sumdf = pd.DataFrame()
    confdf = pd.DataFrame()
    for classification in [
        'RM_vs_RI_vs_LM_vs_LI',
        'RM_vs_RI',
        'LM_vs_LI',
        'RM&RI_vs_LM&LI'
    ]:
        file_name = f'{classification}_B1-B8_pred.pkl'
        class_names = file_name[:-15].split('_vs_')

        try:
            predictions = read_pkl(
                os.path.join(path, 'Predictions', file_name)
            )
        except FileNotFoundError:
            warn(f'File does not exist: {os.path.join(path, "Predictions", file_name)}')
            continue
        pp = PredictionsParser(predictions.y_true, predictions.y_p, class_names)
        print()
        
        sumdf = pd.concat([
                sumdf,
                pp.summary(),
                pd.DataFrame([None for _ in range(pp.summary().shape[0])], index=pp.summary().index),
            ],
            axis=1
        )
        print(pp.confusion)
        confdf = pd.concat([
                confdf,
                pp.confusion,
                pd.DataFrame([None for _ in range(pp.confusion.shape[0])], index=pp.confusion.index),
            ],
            axis=1
        )
        # ppdf = pd.concat([ppdf, pp.confusion], axis=1)
    # print(confdf)
        confdf.to_excel(os.path.join(perf_tables_path, f'{subject_name}_{classification}.xls'))


           Predicted RM  Predicted RI  Predicted LM  Predicted LI
Actual RM            19            11             0             2
Actual RI            11            33             1             1
Actual LM             0             1            25            21
Actual LI             0             0             7            20

           Predicted RM  Predicted RI
Actual RM            31            14
Actual RI            11            20

           Predicted LM  Predicted LI
Actual LM            24             8
Actual LI             8            35



/home/user/Projects/FingerMovementDecoder/utils/machine_learning/confusion.py:302: RuntimeWarning: divide by zero encountered in double_scalars
  self._dor = self.lr_plus / self.lr_minus
/home/user/Projects/FingerMovementDecoder/utils/machine_learning/confusion.py:300: RuntimeWarning: divide by zero encountered in double_scalars
  self._lr_plus = self.tpr / self.fpr
/home/user/Projects/FingerMovementDecoder/utils/machine_learning/confusion.py:302: RuntimeWarning: divide by zero encountered in double_scalars
  self._dor = self.lr_plus / self.lr_minus
/home/user/Projects/FingerMovementDecoder/utils/machine_learning/confusion.py:300: RuntimeWarning: divide by zero encountered in double_scalars
  self._lr_plus = self.tpr / self.fpr
/home/user/Projects/FingerMovementDecoder/utils/machine_learning/confusion.py:302: RuntimeWarning: divide by zero encountered in double_scalars
  self._dor = self.lr_plus / self.lr_minus
/home/user/Projects/FingerMovementDecoder/utils/machine_learning/confusion.

              Predicted RM&RI  Predicted LM&LI
Actual RM&RI               76                1
Actual LM&LI                0               75
                    RM        RI        LM        LI     0        RM  \
Total               30        45        33        44  None        42   
Accuracy      0.842105  0.835526  0.802632  0.796053  None  0.671053   
Specificity   0.908333  0.886792   0.92381     0.808  None  0.645161   
Sensitivity    0.59375  0.717391  0.531915  0.740741  None  0.688889   
Actual RM&RI       NaN       NaN       NaN       NaN   NaN       NaN   
Actual LM&LI       NaN       NaN       NaN       NaN   NaN       NaN   

                    RI     0        LM        LI     0     RM&RI     LM&LI  \
Total               34  None        32        43  None        76        76   
Accuracy      0.671053  None  0.786667  0.786667  None  0.993421  0.993421   
Specificity   0.688889  None  0.813953      0.75  None       1.0  0.987013   
Sensitivity   0.645161  None      0.75  0.

/home/user/Projects/FingerMovementDecoder/utils/machine_learning/confusion.py:302: RuntimeWarning: divide by zero encountered in double_scalars
  self._dor = self.lr_plus / self.lr_minus
/home/user/Projects/FingerMovementDecoder/utils/machine_learning/confusion.py:300: RuntimeWarning: divide by zero encountered in double_scalars
  self._lr_plus = self.tpr / self.fpr
/home/user/Projects/FingerMovementDecoder/utils/machine_learning/confusion.py:302: RuntimeWarning: divide by zero encountered in double_scalars
  self._dor = self.lr_plus / self.lr_minus
/home/user/Projects/FingerMovementDecoder/utils/machine_learning/confusion.py:300: RuntimeWarning: divide by zero encountered in double_scalars
  self._lr_plus = self.tpr / self.fpr
/home/user/Projects/FingerMovementDecoder/utils/machine_learning/confusion.py:302: RuntimeWarning: divide by zero encountered in double_scalars
  self._dor = self.lr_plus / self.lr_minus
/home/user/Projects/FingerMovementDecoder/utils/machine_learning/confusion.

              Predicted RM&RI  Predicted LM&LI
Actual RM&RI               69                1
Actual LM&LI                0               75
                    RM        RI        LM        LI     0        RM  \
Total               31        41        35        38  None        51   
Accuracy      0.793103       0.8  0.848276       0.8  None  0.643836   
Specificity   0.911765  0.826087  0.915094  0.848214  None      0.45   
Sensitivity   0.511628       0.7  0.666667  0.636364  None  0.878788   
Actual RM&RI       NaN       NaN       NaN       NaN   NaN       NaN   
Actual LM&LI       NaN       NaN       NaN       NaN   NaN       NaN   

                    RI     0        LM        LI     0     RM&RI     LM&LI  \
Total               22  None        36        36  None        69        76   
Accuracy      0.643836  None  0.680556  0.680556  None  0.993103  0.993103   
Specificity   0.878788  None   0.69697  0.666667  None       1.0  0.985714   
Sensitivity       0.45  None  0.666667   0

...

/home/user/Projects/FingerMovementDecoder/utils/machine_learning/confusion.py:302: RuntimeWarning: divide by zero encountered in double_scalars
  self._dor = self.lr_plus / self.lr_minus
/home/user/Projects/FingerMovementDecoder/utils/machine_learning/confusion.py:300: RuntimeWarning: divide by zero encountered in double_scalars
  self._lr_plus = self.tpr / self.fpr
/home/user/Projects/FingerMovementDecoder/utils/machine_learning/confusion.py:302: RuntimeWarning: divide by zero encountered in double_scalars
  self._dor = self.lr_plus / self.lr_minus
/home/user/Projects/FingerMovementDecoder/utils/machine_learning/confusion.py:300: RuntimeWarning: divide by zero encountered in double_scalars
  self._lr_plus = self.tpr / self.fpr
/home/user/Projects/FingerMovementDecoder/utils/machine_learning/confusion.py:300: RuntimeWarning: divide by zero encountered in double_scalars
  self._lr_plus = self.tpr / self.fpr
/home/user/Projects/FingerMovementDecoder/utils/machine_learning/confusion.py:30

           Predicted RM  Predicted RI  Predicted LM  Predicted LI
Actual RM            31             6             0             0
Actual RI             7            25             0             0
Actual LM             1             0            36            10
Actual LI             1             1             7            32

           Predicted RM  Predicted RI
Actual RM            31             8
Actual RI             5            34

           Predicted LM  Predicted LI
Actual LM            38            11
Actual LI             8            22

              Predicted RM&RI  Predicted LM&LI
Actual RM&RI               75                0
Actual LM&LI                0               82
                    RM        RI        LM        LI     0        RM  \
Total               40        32        43        42  None        36   
Accuracy      0.904459  0.910828   0.88535  0.878981  None  0.833333   
Specificity      0.925     0.944  0.936364  0.913793  None  0.871795   
Sensitivit

/home/user/Projects/FingerMovementDecoder/utils/machine_learning/confusion.py:300: RuntimeWarning: divide by zero encountered in double_scalars
  self._lr_plus = self.tpr / self.fpr
/home/user/Projects/FingerMovementDecoder/utils/machine_learning/confusion.py:300: RuntimeWarning: divide by zero encountered in double_scalars
  self._lr_plus = self.tpr / self.fpr
/home/user/Projects/FingerMovementDecoder/utils/machine_learning/confusion.py:300: RuntimeWarning: divide by zero encountered in double_scalars
  self._lr_plus = self.tpr / self.fpr
/home/user/Projects/FingerMovementDecoder/utils/machine_learning/confusion.py:302: RuntimeWarning: divide by zero encountered in double_scalars
  self._dor = self.lr_plus / self.lr_minus
/home/user/Projects/FingerMovementDecoder/utils/machine_learning/confusion.py:300: RuntimeWarning: divide by zero encountered in double_scalars
  self._lr_plus = self.tpr / self.fpr
/home/user/Projects/FingerMovementDecoder/utils/machine_learning/confusion.py:302: Ru

           Predicted RM  Predicted RI  Predicted LM  Predicted LI
Actual RM            24            13             1             1
Actual RI             9            26             0             1
Actual LM             4             4            23            12
Actual LI             1             1             9            21

           Predicted RM  Predicted RI
Actual RM            30            10
Actual RI            13            23

           Predicted LM  Predicted LI
Actual LM            29            12
Actual LI             7            26

              Predicted RM&RI  Predicted LM&LI
Actual RM&RI               66                1
Actual LM&LI                0               83
                    RM        RI        LM        LI     0        RM  \
Total               38        44        33        35  None        43   
Accuracy      0.806667  0.813333       0.8  0.833333  None  0.697368   
Specificity   0.873874  0.842105  0.906542  0.881356  None  0.638889   
Sensitivit

/home/user/Projects/FingerMovementDecoder/utils/machine_learning/confusion.py:300: RuntimeWarning: divide by zero encountered in double_scalars
  self._lr_plus = self.tpr / self.fpr
/home/user/Projects/FingerMovementDecoder/utils/machine_learning/confusion.py:302: RuntimeWarning: divide by zero encountered in double_scalars
  self._dor = self.lr_plus / self.lr_minus
/home/user/Projects/FingerMovementDecoder/utils/machine_learning/confusion.py:300: RuntimeWarning: divide by zero encountered in double_scalars
  self._lr_plus = self.tpr / self.fpr


In [4]:
subject_name = 'Ga_Fed_06'

path = f'../Source/Subjects/{subject_name}/LFRNN'

# file_name = 'LM&LI_vs_RM&RI_B1-B8_pred.pkl'
# file_name = 'LM_vs_LI_vs_RM_vs_RI_B1-B8_pred.pkl'
file_name = 'LM_vs_LI_B1-B8_pred.pkl'
class_names = file_name[:-15].split('_vs_')


predictions = read_pkl(
    os.path.join(path, 'Predictions', file_name)
)
print(predictions.y_true.shape)
pp = PredictionsParser(predictions.y_true, predictions.y_p, class_names)
print(pp.accuracy)
print(pp.confusion)
pp.summary()

(75, 2)
0.8266666666666667
           Predicted LM  Predicted LI
Actual LM            28             6
Actual LI             7            34


,LM,LI
Total,35,40
Accuracy,0.826667,0.826667
Specificity,0.829268,0.823529
Sensitivity,0.823529,0.829268


In [193]:



subjects_path = '../Source/Subjects/'
file_name = 'LM_vs_LI_vs_RM_vs_RI_B1-B8_pred.pkl'
# file_name = 'LM&LI_vs_RM&RI_B1-B8_pred.pkl'
class_names = file_name[:-15].split('_vs_')

for subject_name in os.listdir(subjects_path):
    
    if subject_name == 'Pse_Udo':
        continue
    
    subject_path = os.path.join(subjects_path, subject_name)

    predictions = read_pkl(
        os.path.join(subject_path, 'Predictions', file_name)
    )

    pp = PredictionsParser(predictions.y_true, predictions.y_p, class_names)
    print(subject_name)
    print(f'Total accuracy: {pp.accuracy : .2f}')
    summary = pp.summary()
    summary = summary.astype(float).round(2)
    print(summary)
    print(pp.confusion)
    print('-'*100)

Ga_Fed_06
Total accuracy:  0.55
                LM     LI     RM    RI
Total        16.00  12.00  16.00  9.00
Accuracy      0.77   0.77   0.77  0.77
Specificity   0.84   0.89   0.80  0.86
Sensitivity   0.62   0.50   0.67  0.33
           Predicted LM  Predicted LI  Predicted RM  Predicted RI
Actual LM            10             4             1             1
Actual LI             6             8             1             1
Actual RM             0             0             8             4
Actual RI             0             0             6             3
----------------------------------------------------------------------------------------------------
Fe_To_08
Total accuracy:  0.57
                LM     LI     RM    RI
Total        12.00  18.00  16.00  8.00
Accuracy      0.78   0.76   0.80  0.81
Specificity   0.92   0.78   0.81  0.93
Sensitivity   0.50   0.69   0.73  0.42
           Predicted LM  Predicted LI  Predicted RM  Predicted RI
Actual LM             9             7             

/tmp/ipykernel_19962/2523103007.py:30: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if np.any(classes_true != classes_pred):
/home/user/Projects/FingerMovementDecoder/utils/machine_learning/confusion.py:255: RuntimeWarning: invalid value encountered in long_scalars
  self._ppv = self.tp / (self.tp + self.fp)
/home/user/Projects/FingerMovementDecoder/utils/machine_learning/confusion.py:263: RuntimeWarning: invalid value encountered in true_divide
  self._mcc = (self.tp * self.tn - self.fp * self.fn) / \


In [192]:
pp.confusion

,Predicted LM,Predicted LI,Predicted RM,Predicted RI
Actual LM,6,4,3,1
Actual LI,3,2,2,3
Actual RM,0,2,10,1
Actual RI,3,0,7,6


In [196]:
subjects_path = '../Source/Subjects/'
# file_name = 'LM_vs_LI_vs_RM_vs_RI_B1-B8_pred.pkl'
# file_name = 'LM&LI_vs_RM&RI_B1-B8_pred.pkl'
file_name = 'LM_vs_LI_B1-B8_pred.pkl'
class_names = file_name[:-15].split('_vs_')

for subject_name in os.listdir(subjects_path):
    
    if subject_name == 'Pse_Udo':
        continue
    
    subject_path = os.path.join(subjects_path, subject_name)

    predictions = read_pkl(
        os.path.join(subject_path, 'Predictions', file_name),
    )

    y_t = one_hot_decoder(predictions.y_true)

    y_p = one_hot_decoder(predictions.y_p)

    print(subject_name)
    tn, fp, fn, tp = sm.confusion_matrix(y_t, y_p).ravel()
    ce = ConfusionEstimator(tp, tn, fp, fn)

    print(
        f'\tT\tF\n'\
        f'P\t{tp}\t{fp}\n'\
        f'N\t{tn}\t{fn}\n'
        f'Accuracy: {ce.acc}\n'\
        f'Specificity: {ce.spec}\n'\
        f'Sensitivity: {ce.sens}\n'
    )

Ga_Fed_06


ValueError: not enough values to unpack (expected 4, got 1)